# Notebook to test Llama2 performance

# Setup and data cleaning

## Change logs settings

In [ ]:
import pandas as pd
import numpy as np

from datasets import disable_progress_bar
disable_progress_bar() # Disable the "Map" progress bar during the tests

## Load and clean the dataset

This dataset is not on the GitHub repository.
It's composed of work experienced fetched from LinkedIn and labelled between 0 and 4 (0 if it's not related to AI and 4 if it is)

In [ ]:
dataFrame = pd.read_pickle(r'../data/7587_corrige.pkl')
subset = dataFrame[['jobTitle', 'description', 'label']].copy()

subset.reset_index(drop=True, inplace=True)
subset.replace('', np.nan, inplace=True) # drop NaN labels, job titles and descriptions
subset.dropna(inplace=True)

subset['text'] = subset['jobTitle'] + ' ' + subset['description']
subset = subset[['text','label']]
subset_label_transform = subset.copy()

subset_label_transform['label'] = np.where((subset_label_transform["label"] < 3), 0, 1)
subset_label_transform

## Split the dataset in two subsets : the training and test sets

In [ ]:
from benchmark.utility import split_dataset
train_set, test_set = split_dataset(subset_label_transform, 0.2)

## Run tests

In [ ]:
from benchmark.utility import save_to_json
from benchmark.tests import n_shot_tests
from benchmark.train_eval_task import llama2_f1_score

### N-shots

In [ ]:
params = {
    "n_shot": [8],
    "n_iter": 1,
    "n_max_iter_per_shot": 1,
    "model": "meta-llama/Llama-2-7b-chat-hf",
    "loss": "UNKNOWN"
}

results, train_times, eval_times = n_shot_tests(params, train_set, test_set, few_shot_model_f1_function=llama2_f1_score)

save_to_json(results, train_times, eval_times, params,  r'../results/llama2/n_shot')